# Design Docs for New CMC

### Current Problems

- No test code
- No formatter, linter
- Too many arguments
- Only few combinations of the arguments works OK
- T is one dimensional while X is two
- Examples are not good
- Dual SGD is not supported
- Kernel mismatch between QB and KCMC
- Unstable continuous treatment model
- No consistency plot
- No doubly robust
- No information criterion
- No cross-validation
- No hypothesis testing
- It should take both torch tensor and numpy array as input
- Avoid type 3 font of matplotlib at all cost
- Avoid space in the file names of plots so that it is easier to submit to arxiv

### Where to reduce the redundancy?
- choice of kernel (use the QB scaling)
- no intersection of uncertainty set
- no Hajek constraints
- provide true p_t (with a good example) or estimate it independently
- no normalize_p_t (expricitly provide normalized one if necessary)
  - ignore fractional programming if possible
- no testing for policy (test on the difference of policy value or doubly-robust method is more interesting)
- no policy difference for policy improvement (or provide wrapper)
- only assume R or [K] as the action space
- no support for upper bound
- only use torch.Tensor as an input
  - torch.set_default_dtype(float)
  - do not use numpy for creating new tensor
- do not support dual with limited domain (e.g. v < 1.0)

### Package Design: Create Multiple Classes
- BasePolicy
- BaseCMCEstimator
  - fit
  - predict_lower_bound
  - predict_lower_bound_dual
- KCMCEstimator
  - attributes: problem, constraints, if there is
  - Primal, CVXPY, Hessian, GIC, Hypothesis Testing (asymptotics, confint).
- ~DRKCMCEstimator?~
- GPKCMCEstimator
  - fit only (for simplicity)
- DualKCMCEstimator
  - fit (SGD), use Nystrom with KPCA
- DualNCMCEstimator
  - fit (SGD)
- IPWEstimator
- HajekEstimator
- ZSB Estimator
- k_fold_cv, 
- test_*.py
- type it with mypy
- add docstring
- formatter
- ~maybe use jax for Hessian and future use~ We only need an average of Hessian, which is Hessian of the average

### What to do?
- Read DR-paper
  - See how important DR is
  - Understand how they apply asymptotics
  - DR might reduce the variance in trace calculation
  - Read it through just in case I missed something important
- Read Minmax-optimal Policy Learning ...
  - understand the feasibility of sensitivity analysis of propensity score
- Implement good examples
- Start implementing the package
- Make function and attiributes private

### Bonus:
- i.e. Not essential for publication in journal
- Mixed policy testing
  - implement for box constraints, whose dual does not depend on eta_f
- Understand the realizability results 
  - important for numerical example at least
  - can make generalized proof at best

In [ ]:
import numpy as np
import cvxpy as cp

In [13]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T @ P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(x, P) + q.T @ x), [G @ x <= h, A @ x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)


The optimal value is 86.891415855699
A solution x is
[-1.68244521  0.29769913 -2.38772183 -2.79986015  1.18270433 -0.20911897
 -4.50993526  3.76683701 -0.45770675 -3.78589638]
A dual solution corresponding to the inequality constraints is
[ 0.          0.          0.          0.          0.         10.45538054
  0.          0.          0.         39.67365045  0.          0.
  0.         20.79927156  6.54115873]


In [14]:
ineq, eq = prob.constraints

In [15]:
ineq.dual_value

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       10.45538054,  0.        ,  0.        ,  0.        , 39.67365045,
        0.        ,  0.        ,  0.        , 20.79927156,  6.54115873])

## Try cp.perspective

In [23]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
s = cp.Variable(1, nonneg=True)
f = -cp.sum(cp.log(x))
persp = cp.perspective(f, s)
prob = cp.Problem(cp.Minimize(cp.sum(persp)), [G @ x <= h, A @ x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)


The optimal value is inf
A solution x is
None
A dual solution corresponding to the inequality constraints is
None
